# **Arxiv metadata Analytics with PySpark RDD: JSON case study**

### Udemy Course: Best Hands-on Big Data Practices and Use Cases using PySpark

### Author: Amin Karami (PhD, FHEA)
#### email: amin.karami@ymail.com

In [ ]:
########## ONLY in Colab ##########
!pip3 install pyspark
########## ONLY in Colab ##########

In [1]:
########## ONLY in Ubuntu Machine ##########
# Load Spark engine
#!pip3 install -q findspark
import findspark
findspark.init()
########## ONLY in Ubuntu Machine ##########

In [2]:
# Initializing Spark
from pyspark import SparkContext, SparkConf
conf1 = SparkConf().setAppName("Archive_PySpark").setMaster("local[*]")
sc = SparkContext(conf = conf1)
print(sc)

<SparkContext master=local[*] appName=Archive_PySpark>


In [3]:
# Read and Load Data to Spark
# Data source: https://www.kaggle.com/Cornell-University/arxiv/version/62
import json
#we want to read data by attribute, not line by line. And divide it to 100 partitions.
rdd_json = sc.textFile("json_data/arxiv-metadata-oai-snapshot.json", 100)
#with below line, we say spark read it as a json file, don't read iy line by line
rdd = rdd_json.map(lambda x: json.loads(x))
rdd.persist()

PythonRDD[2] at RDD at PythonRDD.scala:53

In [4]:
# Check the number of parallelism and partitions:
print(sc.defaultParallelism)
print(rdd.getNumPartitions())

8
100


## Question 1: Count elements

In [5]:
rdd.count()
#go check from http://localhost:4040/

2011231

## Question 2: Get the first two records


In [6]:
rdd.take(2)

[{'id': '0704.0001',
  'submitter': 'Pavel Nadolsky',
  'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan",
  'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies',
  'comments': '37 pages, 15 figures; published version',
  'journal-ref': 'Phys.Rev.D76:013009,2007',
  'doi': '10.1103/PhysRevD.76.013009',
  'report-no': 'ANL-HEP-PR-07-12',
  'categories': 'hep-ph',
  'license': None,
  'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with d

## Question 3: Get all attributes


In [14]:
rdd.flatMap(lambda x : x.keys()).distinct().collect()

['authors',
 'comments',
 'title',
 'id',
 'journal-ref',
 'versions',
 'submitter',
 'categories',
 'update_date',
 'authors_parsed',
 'report-no',
 'license',
 'abstract',
 'doi']

## Question 4: Get the name of the licenses

In [15]:
rdd.map(lambda x : x["license"]).distinct().collect()

[None,
 'http://creativecommons.org/licenses/publicdomain/',
 'http://creativecommons.org/licenses/by-nc-nd/4.0/',
 'http://creativecommons.org/licenses/by-nc-sa/4.0/',
 'http://creativecommons.org/licenses/by-nc-sa/3.0/',
 'http://creativecommons.org/licenses/by/3.0/',
 'http://creativecommons.org/licenses/by/4.0/',
 'http://creativecommons.org/publicdomain/zero/1.0/',
 'http://arxiv.org/licenses/nonexclusive-distrib/1.0/',
 'http://creativecommons.org/licenses/by-sa/4.0/']

## Question 5: Get the shortest and the longest titles

In [17]:
shortest_rdd_map = rdd.map(lambda x: x["title"]).reduce(lambda x, y : x if x<y else y )
longest_rdd_map = rdd.map(lambda x: x["title"]).reduce(lambda x, y : x if x>y else y )
print("The shorttest:",shortest_rdd_map)
print("The longest:",longest_rdd_map)

The shorttest: !-Graphs with Trivial Overlap are Context-Free
The longest: Weyl formula for the negative dissipative eigenvalues of Maxwell's
  equations


## Question 6: Find abbreviations with 5 or more letters in the abstract

In [19]:
import re
def get_abbreviations(line):
    return re.search(r"\(([A-Za-z][^_ /\\<>]{5,})\)",line)
    if result:
        return result.group(1)
rdd.filter(lambda x : get_abbreviations(x["abstract"])).count()

192721

## Question 7: Get the number of archive records per month ('update_date' attribute)

In [21]:
import datetime
def extract_date(DateIn):
    d = datetime.datetime.strptime(DateIn,"%Y-%m-%d")
    return d.month
rdd.map(lambda x : (extract_date(x["update_date"]), 1)).reduceByKey(lambda x,y : x+y).sortBy(lambda l: l[1]).collect()

[(2, 116948),
 (4, 117126),
 (7, 122649),
 (3, 126458),
 (12, 132305),
 (1, 134247),
 (8, 138469),
 (9, 138978),
 (6, 191746),
 (10, 197755),
 (5, 296587),
 (11, 297963)]

## Question 8: Get the average number of pages

In [ ]:
def get_Page(line):
    search = re.findall('\d+ pages' , line)
    if search:
        return int(search[0].split(" ")[0])
    else:
        return 0
rdd_average = rdd.map(lambda x: get_Page(x["comments"] if x["comments"] != None else "None"))
#remove 0
rdd_average = rdd_average.filter(lambda x : x != 0)
average_counter = rdd_average.count()
average_summation = rdd_average.reduce(lambda x,y : x+y)
print(average_counter)
print(average_summation)
print("average of pages is ",(average_summation/average_counter))